# Event Display Hits Generator

_Authors: Jurjan Bootsma, Bouke Jung, Maarten de Jong_

This jupyter notebook contains an event display generator which looks at the hits as a function of time. You can choose a flavour to look at and you can choose which type of plot you want to look at (3D, 2D or polar). Additionally, you can use timesliders to look at different moments to see which PMTs might be triggered. The first part has a generator for each flavour and the last part compares them all at once.

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import km3pipe as kp
import km3modules as km
import km3io
from km3net_testdata import data_path
from particle import Particle
import matplotlib.animation as animation
import ipywidgets as iw
import IPython
from IPython.display import display

In [2]:
#Reading the data  
    
file_electron = "/dcache/km3net/mc/atm_neutrino/KM3NeT_ORCA_115_20m_9m/v5.0/JPPv16/reco/mcv5.0.gsg_elec-CC_1-10GeV.km3sim.JDK.jte.jmergefit.orca.709.root" 
file_muon = "/dcache/km3net/mc/atm_neutrino/KM3NeT_ORCA_115_20m_9m/v5.0/JPPv16/reco/mcv5.0.gsg_muon-CC_1-10GeV.km3sim.JDK.jte.jshowerchain.orca.821.root" 
file_tau = "/dcache/km3net/mc/atm_neutrino/KM3NeT_ORCA_115_20m_9m/v5.0/JPPv16/reco/mcv5.0.gsg_tau-CC_3.4-10GeV.km3sim.JDK.jte.jshowerchain.orca.aanet.474.root"


# Functions

In [3]:
def lepton_finder(pdg_ids):
    electron_bool = (np.abs(pdg_ids) == 11)
    muon_bool = (np.abs(pdg_ids) == 13)
    tau_bool = (np.abs(pdg_ids) == 15)
    lepton_bool = np.array(electron_bool+muon_bool+tau_bool)
    return lepton_bool

In [4]:
def particles_momentum(f_event):
    vectors_final = rotation_particles(f_event)
    
    momentum = get_momentum(f_event.mc_tracks)
    
    vectors_final[0] = vectors_final[0]*momentum
    vectors_final[1] = vectors_final[1]*momentum
    vectors_final[2] = vectors_final[2]*momentum
    
    return vectors_final

In [5]:
def polar(x, y, z):
    r = np.sqrt(x**2+y**2+z**2)
    theta = np.arccos(z/r)
    theta = np.nan_to_num(theta)
    
    y_mask1 = (y>=0)
    
    phi1 = np.arccos(x/np.sqrt(x**2+y**2))
    phi_array1 = phi1*y_mask1
       
    y_mask2 = (y<0)
    phi2 = 2*np.pi - np.arccos(x/np.sqrt(x**2+y**2))
    phi_array2 = phi2*y_mask2
    
    phi = phi_array1+phi_array2
    phi = np.nan_to_num(phi)
    
    return r, theta, phi

In [6]:
def rotation_particles(f_event):
    pos = tuple(np.zeros(f_event.n_mc_trks))
    dir_x = tuple(f_event.mc_tracks.dir_x)
    dir_y = tuple(f_event.mc_tracks.dir_y)
    dir_z = tuple(f_event.mc_tracks.dir_z)
    
    vector = np.array([dir_x[0], dir_y[0], dir_z[0]])
    
    vector_without_y = np.array([vector[0], 0, vector[2]])    
    angle_from_z = kp.math.angle(vector_without_y, np.array([1,0,0]))
    y_axis = np.array([0,1,0])
    
    if vector[2] < 0:
        angle_from_z = (2*np.pi-angle_from_z)
    rotation_matrix_from_z = kp.math.rotation_matrix(y_axis,angle_from_z)
    
    vector_new = rotation_matrix_from_z@vector
    
    vector_without_z = np.array([vector_new[0], vector_new[1], 0])
    angle_from_y = kp.math.angle(vector_without_z, np.array([1,0,0]))
    z_axis = np.array([0,0,1])
    
    if vector_new[1] > 0:
        angle_from_y = (2*np.pi-angle_from_y)
    rotation_matrix_from_y = kp.math.rotation_matrix(z_axis,angle_from_y)
    
    vector_final = rotation_matrix_from_y@vector_new
    
    vectors = np.array([dir_x, dir_y, dir_z])
    vectors_new = rotation_matrix_from_z@vectors
    vectors_final = rotation_matrix_from_y@vectors_new
    
    return vectors_final

In [7]:
def get_momentum(mc_tracks):
    masses = get_masses(mc_tracks) #MeV
    Es = mc_tracks.E #GeV
    Es_MeV = Es * 10**3 #MeV
    momentum = np.sqrt(Es_MeV**2 - masses**2)
    momentum = np.nan_to_num(momentum) * 10**-3
    return momentum

In [8]:
def get_masses(mc_tracks):
    pdg_ids = mc_tracks.pdgid
    masses = np.zeros(len(pdg_ids))
    i=0
    for pdg in pdg_ids:
        p = Particle.from_pdgid(pdg)
        mass = p.mass
        if mass == None:
            mass = 0
        masses[i]=mass
        i=i+1
    return masses

In [9]:
def rotate(vector, vectors):
    vector_without_y = np.array([vector[0], 0, vector[2]])    
    angle_from_z = kp.math.angle(vector_without_y, np.array([1,0,0]))
    y_axis = np.array([0,1,0])
    
    if vector[2] < 0:
        angle_from_z = (2*np.pi-angle_from_z)
    rotation_matrix_from_z = kp.math.rotation_matrix(y_axis,angle_from_z)
    
    vector_new = rotation_matrix_from_z@vector
    
    vector_without_z = np.array([vector_new[0], vector_new[1], 0])
    angle_from_y = kp.math.angle(vector_without_z, np.array([1,0,0]))
    z_axis = np.array([0,0,1])
    
    if vector_new[1] > 0:
        angle_from_y = (2*np.pi-angle_from_y)
    rotation_matrix_from_y = kp.math.rotation_matrix(z_axis,angle_from_y)
    
    vector_final = rotation_matrix_from_y@vector_new

    vectors_new = rotation_matrix_from_z@vectors
    vectors_final = rotation_matrix_from_y@vectors_new
    
    return vectors_final

In [10]:
def DMT_pos_from_id(pmt_ids):
    detx = data_path("detx/orca_115strings_av23min20mhorizontal_18OMs_alt9mvertical_v1.detx")
    det = kp.hardware.Detector(detx)
    
    x_array = np.zeros(len(pmt_ids))
    y_array = np.zeros(len(pmt_ids))
    z_array = np.zeros(len(pmt_ids))
    
    i=0
    for ids in pmt_ids:
        pmt = det.pmts[det.pmts.pmt_id == ids]
        x = pmt.pos_x
        y = pmt.pos_y
        z = pmt.pos_z
        x_array[i]=x
        y_array[i]=y
        z_array[i]=z      
        i=i+1
    return x_array, y_array, z_array

In [11]:
def hit_arrays(f_event, steps=20):
    neutrino_dir = np.array([f_event.mc_tracks.dir_x[0],f_event.mc_tracks.dir_y[0],f_event.mc_tracks.dir_z[0]])
    
    pos_x = f_event.mc_tracks.pos_x[0]
    pos_y = f_event.mc_tracks.pos_y[0]
    pos_z = f_event.mc_tracks.pos_z[0]    
    
    pmt_ids = f_event.mc_hits.pmt_id
    time = f_event.mc_hits.t

    x,y,z = DMT_pos_from_id(pmt_ids)
    pos = np.array([x,y,z])
    pos_PMTs = rotate(neutrino_dir, pos)
    
    x,y,z = pos_PMTs[0], pos_PMTs[1], pos_PMTs[2]
    
    video = np.linspace(np.min(time)-30,np.max(time)+30,steps)
    x_array_t = np.zeros((steps,len(pmt_ids)))
    y_array_t = np.zeros((steps,len(pmt_ids)))
    z_array_t = np.zeros((steps,len(pmt_ids)))
    for i in range(steps):
        time_bool1 = (time > video[i-1])
        time_bool2 = (time < video[i])
        xi = x*time_bool1*time_bool2
        yi = y*time_bool1*time_bool2
        zi = z*time_bool1*time_bool2
        x_array_t[i] = xi
        y_array_t[i] = yi
        z_array_t[i] = zi
        
    return x_array_t,y_array_t,z_array_t
    
    


# Single Flavour

In [12]:
opts = dict(continuous_update=False)
@iw.interact
def interactive_hits(event=iw.BoundedIntText(value=0,min=0,max=100,description="Event"),
                     filename=iw.Dropdown(options=[("Electron",file_electron),("Muon",file_muon),
                                                   ("Tau",file_tau)],description="Neutrino"),
                     plot=iw.Dropdown(options=[("3D","3D"),("2D","2D"),
                                               ("Polar","polar"),],
                                      description="Kind of plot")):
    global x_array_t
    global y_array_t
    global z_array_t
    
    f = km3io.OfflineReader(data_path(filename))    
    
    neutrino_dir = np.array([f[event].mc_tracks.dir_x[0],f[event].mc_tracks.dir_y[0],f[event].mc_tracks.dir_z[0]])
    pos_x = f[event].mc_tracks.pos_x[0]
    pos_y = f[event].mc_tracks.pos_y[0]
    pos_z = f[event].mc_tracks.pos_z[0]

    pmt_ids = f[event].mc_hits.pmt_id
    time = f[event].mc_hits.t

    x,y,z = DMT_pos_from_id(pmt_ids)
    pos = np.array([x,y,z])
    pos_PMTs = rotate(neutrino_dir, pos)

    x,y,z = pos_PMTs[0], pos_PMTs[1], pos_PMTs[2]
    

    x_bound = np.max(np.array([np.abs(pos_x-np.max(x)),np.abs(pos_x-np.min(x))]))
    y_bound = np.max(np.array([np.abs(pos_y-np.max(y)),np.abs(pos_y-np.min(y))]))
    z_bound = np.max(np.array([np.abs(pos_z-np.max(z)),np.abs(pos_z-np.min(z))]))

    steps = 20
    video = np.linspace(np.min(time)-30,np.max(time)+30,steps)
    x_array_t = np.zeros((steps,len(pmt_ids)))
    y_array_t = np.zeros((steps,len(pmt_ids)))
    z_array_t = np.zeros((steps,len(pmt_ids)))
    for i in range(steps):
        time_bool1 = (time > video[i-1])
        time_bool2 = (time < video[i])
        xi = x*time_bool1*time_bool2
        yi = y*time_bool1*time_bool2
        zi = z*time_bool1*time_bool2
        x_array_t[i] = xi
        y_array_t[i] = yi
        z_array_t[i] = zi
        
        
    positions = np.array([pos_x,pos_y,pos_z])
    position_neutrino = rotate(neutrino_dir, positions)
    pos_x,pos_y,pos_z = position_neutrino[0], position_neutrino[1], position_neutrino[2]
    
    
    momentum = particles_momentum(f[event])
    
    opts = dict(continuous_update=False)
    @iw.interact
    def interactive_time(t=iw.IntSlider(value=1,min=1,max=20)):
    
        global x_array_t
        global y_array_t
        global z_array_t
        
        t=t-1
        
        
        if plot == "3D":
            fig, ax = plt.subplots(figsize=(12.5,12.5))
            ax = plt.axes(projection='3d')
            ax.set_xlim(pos_x-x_bound, pos_x+x_bound)
            ax.set_ylim(pos_y-y_bound, pos_y+y_bound)
            ax.set_zlim(pos_z-z_bound, pos_z+z_bound)
            ax.view_init(0,180)
            ax.scatter3D(pos_x,pos_y,pos_z,color='red')
            ax.scatter3D((x_array_t[t])[x_array_t[t] !=0],(y_array_t[t])[y_array_t[t]!=0],(z_array_t[t])[z_array_t[t]!=0])
            #ax.quiver(pos_x,pos_y,pos_z,lepton[0],lepton[1],lepton[2], arrow_length_ratio=0)
            plt.show()
    
        elif plot == "2D":
            fig = plt.figure(figsize=(7.5,7.5))
            plt.scatter(pos_y, pos_z, color='red')
            plt.xlim(pos_y-y_bound, pos_y+y_bound)
            plt.ylim(pos_z-z_bound, pos_z+z_bound)
            plt.xlabel("y")
            plt.ylabel("z")
            plt.scatter((y_array_t[t])[y_array_t[t]!=0],(z_array_t[t])[z_array_t[t]!=0])
            plt.show()
    
        elif plot == "polar":
            r_t, theta_t, phi_t = polar(y_array_t[t]-pos_y,z_array_t[t]-pos_z,x_array_t[t]-pos_x)
            r_0, theta_0, phi_0 = polar(-pos_y,-pos_z,-pos_x)
            
            fig, ax = plt.subplots(figsize=(7.5,7.5))
            ax = fig.add_subplot(projection='polar')
            ax.scatter(phi_t[phi_t!=phi_0], r_t[r_t!=r_0],c='blue', s=20)
            plt.show()
    
    

interactive(children=(BoundedIntText(value=0, description='Event'), Dropdown(description='Neutrino', options=(…

# All Flavours Comparison

In [13]:
f_e = km3io.OfflineReader(data_path(file_electron))
f_m = km3io.OfflineReader(data_path(file_muon))
f_t = km3io.OfflineReader(data_path(file_tau))

E_e_sort = np.sort(np.around((f_e.mc_tracks.E)[:,0]))
index_e = np.argsort(np.around((f_e.mc_tracks.E)[:,0]))
E_e_sort2 = E_e_sort[E_e_sort!=1]
E_e_sort3 = E_e_sort2[E_e_sort2!=2]
index_e = index_e[E_e_sort!=1]
index_e = index_e[E_e_sort2!=2]

E_m_sort = np.sort(np.around((f_m.mc_tracks.E)[:,0]))
index_m = np.argsort(np.around((f_m.mc_tracks.E)[:,0]))
E_m_sort2 = E_m_sort[E_m_sort!=1]
E_m_sort3 = E_m_sort2[E_m_sort2!=2]
index_m = index_m[E_m_sort!=1]
index_m = index_m[E_m_sort2!=2]

E_t_sort = np.sort(np.around((f_t.mc_tracks.E)[:,0]))
index_t = np.argsort(np.around((f_t.mc_tracks.E)[:,0]))
E_t_sort2 = E_t_sort[E_t_sort!=1]
E_t_sort3 = E_t_sort2[E_t_sort2!=2]
index_t = index_t[E_t_sort!=1]
index_t = index_t[E_t_sort2!=2]


opts = dict(continuous_update=False)
@iw.interact
def interactive_hits(event=iw.BoundedIntText(value=0,min=0,max=100,description="Event"),
                     energy=iw.IntSlider(value=4,min=4,max=10),
                     plot=iw.Dropdown(options=[("3D","3D"),("2D","2D"),
                                               ("Polar transverse","polar"),
                                               ("3D Momentum","3D Momentum")],
                                      description="Kind of plot")):
    global x_array_t
    global y_array_t
    global z_array_t
    
    
    #Electron
    f_e = km3io.OfflineReader(data_path(file_electron))
    index_e_energy = index_e[E_e_sort3==energy]
    
    neutrino_dir_e = np.array([f_e[index_e_energy[event]].mc_tracks.dir_x[0],
                               f_e[index_e_energy[event]].mc_tracks.dir_y[0],
                               f_e[index_e_energy[event]].mc_tracks.dir_z[0]])
    
    x_array_t_e,y_array_t_e,z_array_t_e = hit_arrays(f_e[index_e_energy[event]])
    
    pos_x_e = f_e[index_e_energy[event]].mc_tracks.pos_x[0]
    pos_y_e = f_e[index_e_energy[event]].mc_tracks.pos_y[0]
    pos_z_e = f_e[index_e_energy[event]].mc_tracks.pos_z[0]
    
    positions = np.array([pos_x_e,pos_y_e,pos_z_e])
    position_neutrino = rotate(neutrino_dir_e, positions)
    pos_x_e,pos_y_e,pos_z_e = position_neutrino[0], position_neutrino[1], position_neutrino[2]
    
    x_array_t_e = x_array_t_e - pos_x_e
    y_array_t_e = y_array_t_e - pos_y_e
    z_array_t_e = z_array_t_e - pos_z_e 
    
    E_e = round(f_e[index_e_energy[event]].mc_tracks.E[0])
    
    momentum_e = particles_momentum(f_e[index_e_energy[event]])
    lepton_bool = lepton_finder(f_e[index_e_energy[event]].mc_tracks.pdgid)
    lepton_index = np.where(lepton_bool)[0][0]
    
    lepton_e = momentum_e[:,lepton_index] * 250
    
    
    #Muon
    f_m = km3io.OfflineReader(data_path(file_muon))
    index_m_energy = index_m[E_m_sort3==energy]
    
    neutrino_dir_m = np.array([f_m[index_m_energy[event]].mc_tracks.dir_x[0],
                               f_m[index_m_energy[event]].mc_tracks.dir_y[0],
                               f_m[index_m_energy[event]].mc_tracks.dir_z[0]])
    
    x_array_t_m,y_array_t_m,z_array_t_m = hit_arrays(f_m[index_m_energy[event]])
    
    pos_x_m = f_m[index_m_energy[event]].mc_tracks.pos_x[0]
    pos_y_m = f_m[index_m_energy[event]].mc_tracks.pos_y[0]
    pos_z_m = f_m[index_m_energy[event]].mc_tracks.pos_z[0]
    
    positions = np.array([pos_x_m,pos_y_m,pos_z_m])
    position_neutrino = rotate(neutrino_dir_m, positions)
    pos_x_m,pos_y_m,pos_z_m = position_neutrino[0], position_neutrino[1], position_neutrino[2]
    
    x_array_t_m = x_array_t_m - pos_x_m
    y_array_t_m = y_array_t_m - pos_y_m
    z_array_t_m = z_array_t_m - pos_z_m
    
    E_m = round(f_m[index_m_energy[event]].mc_tracks.E[0])
    
    momentum_m = particles_momentum(f_m[index_m_energy[event]])
    lepton_bool = lepton_finder(f_m[index_m_energy[event]].mc_tracks.pdgid)
    lepton_index = np.where(lepton_bool)[0][0]
    
    lepton_m = momentum_m[:,lepton_index] * 250
    
    #Tau
    f_t = km3io.OfflineReader(data_path(file_tau))
    index_t_energy = index_t[E_t_sort3==energy]
    
    neutrino_dir_t = np.array([f_t[index_t_energy[event]].mc_tracks.dir_x[0],
                               f_t[index_t_energy[event]].mc_tracks.dir_y[0],
                               f_t[index_t_energy[event]].mc_tracks.dir_z[0]])    
    
    x_array_t_t,y_array_t_t,z_array_t_t = hit_arrays(f_t[index_t_energy[event]])

    
    pos_x_t = f_t[index_t_energy[event]].mc_tracks.pos_x[0]
    pos_y_t = f_t[index_t_energy[event]].mc_tracks.pos_y[0]
    pos_z_t = f_t[index_t_energy[event]].mc_tracks.pos_z[0]
    
    positions = np.array([pos_x_t,pos_y_t,pos_z_t])
    position_neutrino = rotate(neutrino_dir_t, positions)
    pos_x_t,pos_y_t,pos_z_t = position_neutrino[0], position_neutrino[1], position_neutrino[2]
    
    x_array_t_t = x_array_t_t - pos_x_t
    y_array_t_t = y_array_t_t - pos_y_t
    z_array_t_t = z_array_t_t - pos_z_t
    
    E_t = round(f_t[index_t_energy[event]].mc_tracks.E[0])
    
    player = iw.Play(value=1,min=1,max=20,step=1,interval=2000)
    slider = iw.IntSlider(value=1,min=1,max=20,step=1)  
    caption = iw.Label(value='Time')
    l = iw.link((player, 'value'), (slider, 'value'))
            
    opts = dict(continuous_update=False)
    @iw.interact
    def interactive_time(y=player, t=slider):
    
        global x_array_t
        global y_array_t
        global z_array_t
        
        t=t-1

        
        if plot == "3D":
            fig, ax = plt.subplots(figsize=(12.5,12.5))
            ax = plt.axes(projection='3d')
            ax.set_xlim(-200, 200)
            ax.set_ylim(-200, 200)
            ax.set_zlim(-200, 200)
            ax.view_init(0,180)
            ax.scatter3D(0,0,0,color='black')
            ax.scatter3D((x_array_t_e[t])[x_array_t_e[t]!=-pos_x_e],(y_array_t_e[t])[y_array_t_e[t]!=-pos_y_e],(z_array_t_e[t])[z_array_t_e[t]!=-pos_z_e], color='blue',label=f"Electron, E={E_e} GeV")
            ax.scatter3D((x_array_t_m[t])[x_array_t_m[t]!=-pos_x_m],(y_array_t_m[t])[y_array_t_m[t]!=-pos_y_m],(z_array_t_m[t])[z_array_t_m[t]!=-pos_z_m], color='green',label=f"Muon, E={E_m} GeV")
            ax.scatter3D((x_array_t_t[t])[x_array_t_t[t]!=-pos_x_t],(y_array_t_t[t])[y_array_t_t[t]!=-pos_y_t],(z_array_t_t[t])[z_array_t_t[t]!=-pos_z_t], color='red',label=f"Tau, E={E_t} GeV")
            #ax.quiver(0,0,0,lepton_e[0],lepton_e[1],lepton_e[2],arrow_length_ratio = 0.0,color='blue')
            #ax.quiver(0,0,0,lepton_m[0],lepton_m[1],lepton_m[2],arrow_length_ratio = 0.0,color='green')
            #ax.quiver(0,0,0,lepton_t[0],lepton_t[1],lepton_t[2],arrow_length_ratio = 0.0,color='red')
            plt.legend()
            plt.show()
        
        elif plot == "2D":
            fig = plt.figure(figsize=(10, 9))
            plt.scatter(0, 0, color='black')
            plt.xlim(-200, 200)
            plt.ylim(-200,200)
            plt.xlabel("y")
            plt.ylabel("z")
            plt.scatter((y_array_t_e[t])[y_array_t_e[t]!=-pos_y_e],(z_array_t_e[t])[z_array_t_e[t]!=-pos_z_e], color='blue',label=f"Electron, E={E_e} GeV")
            plt.scatter((y_array_t_m[t])[y_array_t_m[t]!=-pos_y_m],(z_array_t_m[t])[z_array_t_m[t]!=-pos_z_m], color='green',label=f"Muon, E={E_m} GeV")
            plt.scatter((y_array_t_t[t])[y_array_t_t[t]!=-pos_y_t],(z_array_t_t[t])[z_array_t_t[t]!=-pos_z_t], color='red',label=f"Tau, E={E_t} GeV")
            plt.legend()
            plt.show()
            
        elif plot == "polar":
            r_e, theta_e, phi_e = polar(y_array_t_e,z_array_t_e,x_array_t_e)
            r_m, theta_m, phi_m = polar(y_array_t_m,z_array_t_m,x_array_t_m)
            r_t, theta_t, phi_t = polar(y_array_t_t,z_array_t_t,x_array_t_t)
            
            r_0_e, theta_0_e, phi_0_e = polar(-f_e[event].mc_tracks.pos_y[0],-f_e[event].mc_tracks.pos_z[0],-f_e[event].mc_tracks.pos_x[0])
            r_0_m, theta_0_m, phi_0_m = polar(-f_m[event].mc_tracks.pos_y[0],-f_m[event].mc_tracks.pos_z[0],-f_m[event].mc_tracks.pos_x[0])
            r_0_t, theta_0_t, phi_0_t = polar(-f_t[event].mc_tracks.pos_y[0],-f_t[event].mc_tracks.pos_z[0],-f_t[event].mc_tracks.pos_x[0])
            
            transverse_e = np.sqrt(y_array_t_e**2 + z_array_t_e**2)
            transverse_m = np.sqrt(y_array_t_m**2 + z_array_t_m**2)
            transverse_t = np.sqrt(y_array_t_t**2 + z_array_t_t**2)
            
            transverse_0_e = np.sqrt(f_e[event].mc_tracks.pos_y[0]**2 + f_e[event].mc_tracks.pos_z[0]**2)
            transverse_0_m = np.sqrt(f_m[event].mc_tracks.pos_y[0]**2 + f_m[event].mc_tracks.pos_z[0]**2)
            transverse_0_t = np.sqrt(f_t[event].mc_tracks.pos_y[0]**2 + f_t[event].mc_tracks.pos_z[0]**2)
            
            fig, ax = plt.subplots(figsize=(7.5,7.5))
            ax = fig.add_subplot(projection='polar')
            ax.scatter((phi_e[t])[phi_e[t]!=phi_0_e], (transverse_e[t])[transverse_e[t]!=transverse_0_e],color='blue',label=f"Electron, E={E_e} GeV")
            ax.scatter((phi_m[t])[phi_m[t]!=phi_0_m], (transverse_m[t])[transverse_m[t]!=transverse_0_m],color='green',label=f"Muon, E={E_m} GeV")
            ax.scatter((phi_t[t])[phi_t[t]!=phi_0_t], (transverse_t[t])[transverse_t[t]!=transverse_0_t],color='red',label=f"Tau, E={E_t} GeV")
            ax.set_rlim(0,150)
            plt.legend()
            plt.show()
            

interactive(children=(BoundedIntText(value=0, description='Event'), IntSlider(value=4, description='energy', m…